# Tiles construction

Cytomine and SLDC modules need to be installed

https://doc.cytomine.com/admin-guide/ce/ce-install

https://github.com/waliens/sldc-cytomine

In [ ]:
import os
import sldc_cytomine
import sldc
from sldc import Segmenter, DispatchingRule, PolygonClassifier, report_timing, StandardOutputLogger, Logger
from sldc.builder import SLDCWorkflowBuilder
from sldc.image import TileTopology, FixedSizeTileTopology
from sldc_cytomine.tile_builder import CytomineTileBuilder
from sldc_cytomine.tile import CytomineTile
from sldc_cytomine.dump import _infer_image_region, _image_from_zone
from sldc_cytomine.autodetect import infer_protocols
from cytomine import Cytomine
from cytomine.models import PropertyCollection, Property, CurrentUser, Project,ImageInstance, Annotation, Term, ProjectCollection,ImageInstanceCollection, AnnotationCollection, TermCollection, Property, AnnotationTerm
import matplotlib.pyplot as plt
import numpy as np 
from skimage import io
from skimage.color import rgb2gray

Connexion to Cytomin

In [ ]:

host = 'http://cytomine.ulb.ovh'
public_key = 'public_key'
private_key = 'private_key'



with Cytomine(host, public_key, private_key) as cytomine:
    me = CurrentUser().fetch()
    

    projects = ProjectCollection().fetch()
    for project in projects:
        proj_id = project.id
    
    phytolithes = Project().fetch(id=proj_id)

    images = ImageInstanceCollection().fetch_with_filter("project", proj_id)
    for image in images:
        print(image.name)
        print(image.id)
        print("Taille de l'image : {} x {}".format(image.height, image.width))
        print("Résolution de l'image: {}".format(image.resolution))
        im_id = image.id

    image = ImageInstance().fetch(id = im_id)




Extraction of the tiles of size 256x256 and an overlap of 0. The Fixed Size Tile Topology is used. 

In [ ]:
tiles_file = "tiles"
if not os.path.exists(tiles_file):
    os.makedirs(tiles_file)

logger = StandardOutputLogger(Logger.INFO)
slide_class, tile_class = infer_protocols(_image_from_zone(im_id))
im = _infer_image_region(im_id, 0, slide_class)
tile_builder = CytomineTileBuilder(working_path="Cache 256", tile_class= tile_class)
topology = TileTopology(image = im, tile_builder=tile_builder, max_width= 256, max_height= 256, overlap= 0)
topology = FixedSizeTileTopology(topology)
iter = topology.__iter__()

for t in iter:
    coord = t.offset
    x = coord[0]
    y = coord[1]
    t_np = t.np_image
    plt.imsave("tiles_path/{}_{}_{}_{}_{}.png".format(im_id, t.offset_x,t.offset_y, t.width, t.height), t_np)


Construction of an image tile at a defined position and size for a defined builder

In [ ]:
def construct_tile(builder, coord, height, width):
    tile = im.tile(builder, coord, height,width)
    print("tile constructed")
    tile_np = tile.np_image
    return tile_np


# Model detection

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import config as tf_config
from tensorflow.keras import layers, models
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math

In [ ]:
def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)
    #Normalization
    image = tf.cast(image, tf.float64) / 255.0  
    return image

def create_dataset(image_paths, annotation_paths):
    images = [load_and_preprocess_image(path) for path in image_paths]
    annotations = [parse_yolo_annotation(path) for path in annotation_paths]
    dataset = tf.data.Dataset.from_tensor_slices((images, annotations))
    return dataset

def preprocess_data(image, annotation):
    # Expand dimensions for the image
    image = tf.expand_dims(image, axis=0)
    
    # Cast annotation to tf.float32
    annotation = tf.cast(annotation, tf.float32)
    
    return image, annotation

Read the annotations and predictions txt files

In [ ]:

def read_annotations(annotation_path):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    annotations = []
    for line in lines:
        values = line.strip().split()
        class_label = int(values[0])
        x, y, width, height = map(float, values[1:])
        annotations.append((class_label, x, y, width, height))

    return annotations

def read_predictions(predictions_path):
    with open(predictions_path, 'r') as f:
        lines = f.readlines()

    boxes = []
    for line in lines:
        data = eval(line)
        x_sldc, y_sldc = map(int, data[0:2])
        x_c, y_c, w, h = map(float, data[2:])
        
        boxes.append([x_sldc, y_sldc, x_c, y_c, w, h])

    return np.array(boxes)

Convert YOLO format to TensorFlow bounding box format (x_center, y_center, width, height)

In [ ]:

def parse_yolo_annotation(annotation_path):
    with open(annotation_path, 'r') as f:
        lines = f.readlines()

    boxes = []
    for line in lines:
        data = line.strip().split()
        class_label = int(data[0])
        x, y, w, h = map(float, data[1:])
        
        boxes.append([x, y, w, h])

    return np.array(boxes)

Allows to draw a Bounding Box representing the annotation on an image.

In [ ]:
def plot_bounding_boxes(image_path, annotations):
    image = plt.imread(image_path)

    fig, ax = plt.subplots(1)
    ax.imshow(image)

    for annotation in annotations:
        print(annotation)
        class_label, x, y, width, height = annotation
        x *= image.shape[1]
        y *= image.shape[0]
        width *= image.shape[1]
        height *= image.shape[0]
        bbox = patches.Rectangle((x - width / 2, y - height / 2), width, height,
                                 linewidth=2, edgecolor='g', facecolor='none')
        ax.add_patch(bbox)

    plt.axis('off')
    plt.show()

In [ ]:
def get_file_paths_from_folder(folder_path, extension):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(extension)]
    return file_paths

Construction of the Model

In [ ]:
# Loading of pre-trained VGG16
base_model = tf.keras.applications.VGG16(
    include_top=False, weights='imagenet', input_shape=(256, 256, 3)
)

# Fiw the weights
base_model.trainable = False

# Adding the regression head
x = layers.Conv2D(32, 3, activation='relu')(base_model.output)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(256, activation='relu')(x)
predictions = layers.Dense(4)(x)  
# Assuming 4 outputs for bounding box coordinates


model = models.Model(inputs=base_model.input, outputs=predictions)

# Loading of the weights 
model.load_weights('best.h5')

Application of the model on the tiles

In [ ]:

image_folder = tiles_file


# Get a list of all image filenames in the folder
image_filenames = [filename for filename in os.listdir(image_folder) if filename.endswith(('.png', '.jpg'))]
for image_filename in image_filenames:
    image_path = os.path.join(image_folder, image_filename)
    
    
#  Text files creation with [0 0 0 0 0] labels for entry in the model
annotations_folder = 'labels'
if not os.path.exists(annotations_folder):
    os.makedirs(annotations_folder)
for image_filename in image_filenames:
    path = os.path.join(image_folder, image_filename.replace('.png', '.txt'))
    with open(path, 'w') as file:
        file.write("0 0 0 0 0 \n")
        file.close()
        
    
image_paths_test = get_file_paths_from_folder(image_folder, extension=(".png", ".jpg"))
annotation_paths_test = get_file_paths_from_folder(annotations_folder, extension=".txt")

predictions_file = 'predictions'
if not os.path.exists(predictions_file):
    os.makedirs(predictions_file)

nbr_waves = math.ceil(len(image_filenames)/1000)


for i in range(nbr_waves):
    start = (i)*1000
    if (start+1000)<= len(image_filenames):
        end = start+1000
    else:
        end = len(image_filenames)
        print("all predictions done")

    dataset = create_dataset(image_paths_test[start:end], annotation_paths_test[start:end])
    dataset = dataset.map(preprocess_data)
    predictions = model.predict(dataset)

    for j, (image, label) in enumerate(dataset):
        name_im = image_filenames[start + j]
        infos = name_im.split('_')
        infos = [int(number) for number in infos if number.isdigit()]
    
        coords = [infos[1], infos[2]]
        save_path = os.path.join(predictions_file, 'predictions_' + str(i) + '.txt')
        
        pred_box = predictions[j]  
        coords.extend(pred_box.tolist())
        with open(save_path , 'a') as file:
            file.write(str(coords) + '\n')
        

## Extraction of the detections

In [ ]:
import skimage
from skimage.filters import sobel_h, sobel_v
from skimage.color import rgb2gray

# Tenenbaum gradient (not the Wes Anderson movie)
def tenenbaum(image):
    sob_x = sobel_h(image)
    sob_y = sobel_v(image)
    sum_sq = sob_x**2 + sob_y**2
    ten = np.sum(sum_sq)
    return ten


In [ ]:

slide_class, tile_class = infer_protocols(_image_from_zone(im_id))
tile_builder = CytomineTileBuilder(working_path="Cache 256", tile_class= tile_class)
image_micro = _infer_image_region(im_id, 0, slide_class)

tot_saved_tiles = 0
tot_blurr_tiles = 0
tot_small_area = 0


for j in range(len(predictions_file)+1):

    # print("WAVE", j)
    
    predictions_path = os.path.join(predictions_file, 'predictions_'+str(j)+'.txt')
    predictions = read_predictions(predictions_path)
    
    detections_path = 'detections_images'
    if not os.path.exists(detections_path): os.makedirs(detections_path)
    

    count_neg_coord = 0
    count_side_zero = 0
    count_small_area = 0
    count_saved_tiles = 0
    count_blur = 0

    for i in range (len(predictions)):
        
        d_test = predictions[i]

        x_sldc = d_test[0]
        y_sldc = d_test[1]

        x_c = d_test[2]*256
        y_c = d_test[3]*256

        w = d_test[4]*256
        h = d_test[5]*256

        area = w*h
        x_d = x_sldc + x_c - w/2
        y_d = y_sldc + y_c - h/2

        # checking negative corrdinates and null width or height
        if x_c<0 or y_c<0:
            count_neg_coord += 1
            continue

        elif int(w)==0 or int(h)==0:
            count_side_zero += 1
            continue
        
        # Condition on the area of the detection to suppress small/empty detections        
        if area < 5000:
            count_small_area += 1
        
        # If condition respected, construction of the tile, twice the size of the detection
        if area >= 5000:
            width_n = 2*w
            height_n = 2*h
            x_n = x_d - round((w/2))
            y_n = y_d - round((h/2))
    
            if x_n < 0:
                x_n = 0
            if y_n < 0 :
                y_n = 0
    
            detect_n = construct_tile(tile_builder, (int(x_n), int(y_n)), int(width_n), int(height_n))
            detect_bw = rgb2gray(detect_n)
            
            # Tenenbaum gradient threshold
            ten = tenenbaum(detect_bw)
            
            if ten >= 20:
                save_path = os.path.join(detections_path, str(int(x_n)) + "_" + str(int(y_n)) +"_from_" + str(int(x_sldc)) + "_" + str(int(y_sldc)) + ".png")
                plt.imsave(save_path, detect_n)
                count_saved_tiles += 1
            else :
                count_blur +=1



    print(count_saved_tiles, " saved detections and ", count_blur, " images suppressed by the Tenenbaum gradient condition")
    
    tot_blurr_tiles += count_blur
    tot_saved_tiles += count_saved_tiles
    tot_small_area += count_small_area
    

print('In total,', tot_saved_tiles, ' tiles were saved, ', tot_small_area,' were deleted because their area < 5000 and ', tot_blurr_tiles,' tiles with an area greater than 5000 were classified as fuzzy.')
    

## Classification of the detections

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import save_img, img_to_array, array_to_img
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img
from collections import defaultdict
from os.path import basename, join, exists, dirname, realpath
import os
import shutil
import matplotlib.pyplot as plt
import joblib
from sklearn.decomposition import PCA
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from umap import UMAP, AlignedUMAP
from tqdm import tqdm
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from urllib.parse import unquote # python 3

In [ ]:

def clean_filename(s, **kwargs):
  '''Given a string that points to a filename, return a clean filename'''
  s = unquote(os.path.basename(s))
  invalid_chars = '<>:;,"/\\|?*[]'
  for i in invalid_chars: s = s.replace(i, '')
  return s

Extraction of the Inception vectors

In [ ]:
image_filenames = [filename for filename in os.listdir(detections_path) if filename.endswith(('.png', '.jpg'))]
image_paths = [os.path.join(detections_path, filename) for filename in image_filenames]
image_paths = sorted(image_paths)

vector_dir = os.path.join(detections_path, 'inception_vectors')
if not os.path.exists(vector_dir): os.makedirs(vector_dir)
base = InceptionV3(include_top=True, weights='imagenet',)
model = Model(inputs=base.input, outputs=base.get_layer('avg_pool').output)

vecs = []
nbr_wave = len(image_filenames)//100
for j in range(nbr_wave+1):
    with tqdm(100) as progress_bar:
        for i in range(100):            
            vector_path = os.path.join(vector_dir, clean_filename(image_paths[i+(j*100)]) + '.npy')
            if os.path.exists(vector_path):
                vec = np.load(vector_path)
            else:
                image = load_img(image_paths[i+(j*100)])
                im = preprocess_input(img_to_array(image.resize((299,299))))
                vec = model.predict(np.expand_dims(im, 0)).squeeze()
                np.save(vector_path, vec)
            vecs.append(vec)
            progress_bar.update(1)


PCA and UMAP application

In [ ]:
vecs = np.array(vecs)

# PCA

w_tot = PCA(n_components=min(100, len(vecs)), random_state= 24).fit_transform(vecs)
print("PCA done")

# UMAP
n_n = 30*(len(detections_path)/5016)

model = UMAP(
    n_neighbors = n_n,
    min_dist = 0.0,
    n_components = 2,
    metric = 'correlation',
    random_state = 24,
    transform_seed = 24
)

print("UMAP reduction done")

z_tot = model.fit(w_tot).embedding_

Application of knn

In [ ]:
classifier_path = "knn_classifier.joblib"
knn_load = joblib.load(classifier_path)

n_labels = knn_load.predict(z_tot)

interest_points = z_tot[n_labels == 0]


# Visualiser les nouvelles données et le cluster d'intérêt
# plt.scatter(z_tot[:, 0], z_tot[:, 1], c=n_labels, cmap='viridis', alpha=0.2)
# plt.scatter(interest_points[:, 0], interest_points[:, 1], c='red', label='cluster of interest', alpha=0.2)
# plt.xlabel('UMAP 1')
# plt.ylabel('UMAP 2')
# plt.legend()
# plt.title('All data classification')
# plt.show()

Copy cluster of interest in a new file

In [ ]:

print(len(image_paths))
print(len(n_labels))
interest_filenames = [image_paths[i] for i in range(len(n_labels)) if n_labels[i]==0]
non_interest_filenames = [image_paths[i] for i in range(len(n_labels)) if n_labels[i]!= 0]

print(len(interest_filenames))
print(len(non_interest_filenames))

interest_folder = 'cluster_of_interest'
if not os.path.exists(interest_folder):
    os.makedirs(interest_folder)

for i in range(len(interest_filenames)):
    filename = interest_filenames[i]
    basename = os.path.basename(filename)
    save_path = os.path.join(interest_folder, basename)
    shutil.copy(filename, save_path)
    

## Pixplot Visualisation

Pixplot must also be installed

https://github.com/YaleDHLab/pix-plot/blob/master/README.md

In [ ]:
import subprocess

command_pixplot = 'pixplot --images "detections/*.png" --n_neighbors 30 --min_dist 0.0'
process_pixplot = subprocess.Popen(command1, shell=True)
process_pixplot.wait() 

command2 = 'python -m http.server 5000'
process2 = subprocess.Popen(command2, shell=True)
process2.wait() 